In [1]:
from bdp_calcs import (
  parsear_tiempos, calc_sueno, calc_estimulantes, calc_actividad_luz_pantalla,
  calc_psicosocial, calc_medicacion_sustancias, calc_indices, calc_temporales
)
import pandas as pd

input_csv ="data/bdp_sample.csv"
targets = "animo,claridad,estres,activacion".split(",")
output ="./outputs"
target_email = "fabianleandro.cv@gmail.com"
row_idx = -1

In [2]:
from bdp_calcs.audit import core_audit
from bdp_calcs.drivers import *
from bdp_calcs.relations import *
from bdp_calcs.normalize import *
from bdp_calcs.ema_report import build_ema_report 

In [3]:
df_forms = pd.read_csv(input_csv, encoding="utf-8", decimal=",")  # o el CSV del forms
df_clean, colmap = normalize_bdp_df(df_forms)
df = build_ema_report(df_clean, target_email, targets)

Saved: output/informeBDP_EMA.html



#targets = [t.strip() for t in targets if t.strip()]
audit_df = core_audit(df, targets)
audit_path = output + "/core_audit.csv"
audit_df.to_csv(audit_path, index=False, encoding="utf-8")
print("Audit guardado en:", audit_path)
print(audit_df)



# Drivers del día para cada target
for t in targets:
    res = drivers_del_dia(df, t, top_k=3)
    txt = res.get("texto","")
    tab = res.get("tabla")
    narrativa = res.get("narrativa")
    print("\n[Drivers del día]", t, "→", txt, "\nInterpretación →", narrativa)
    if isinstance(tab, pd.DataFrame) and not tab.empty:
        tab.to_csv(os.path.join(output, f"{t}_drivers_del_dia.csv"), index=False, encoding="utf-8")


# 1) Wellbeing index
wb = wellbeing_index(df, targets=targets)
wb_path = os.path.join(output, "wellbeing_index_series.csv")
pd.DataFrame({"WB": wb}).to_csv(wb_path, index=False, encoding="utf-8")
print("WB guardado en:", wb_path)

# 2) Correlaciones (niveles y deltas)
cors = core_correlations(df, targets=targets)
cors["levels"].to_csv(os.path.join(output, "core_cor_levels.csv"), encoding="utf-8")
cors["deltas"].to_csv(os.path.join(output, "core_cor_deltas.csv"), encoding="utf-8")
print("Correlaciones listas: core_cor_levels.csv, core_cor_deltas.csv")

# 3) Influencia cruzada (lag-1)
infl = crosslag_influence(df, targets=targets)
if isinstance(infl, pd.DataFrame) and not infl.empty:
    infl.to_csv(os.path.join(output, "core_crosslag_influence.csv"), encoding="utf-8")
    print("Influencia cruzada guardada en: core_crosslag_influence.csv")
else:
    print("Influencia cruzada: insuficiente señal (necesita más filas).")

# 4) Agregación de drivers del día a lo largo de la historia
agg = aggregate_drivers_history(df, drivers_func=drivers_del_dia, targets=targets, top_k=3)
# Exportar por target
for t, ser in agg["avg_contrib"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_avg_contrib.csv"), header=["avg_contrib"], encoding="utf-8")
for t, ser in agg["abs_contrib"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_abs_contrib.csv"), header=["mean_abs_contrib"], encoding="utf-8")
for t, ser in agg["freq_sign"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_freq_pos.csv"), header=["freq_positive"], encoding="utf-8")
print("Agregación de drivers exportada (avg/abs/freq).")

# 5) Sinergia de bienestar (qué drivers tienden a mejorar/empeorar el compuesto)
syn = wellbeing_synergy(agg)
if not syn.empty:
    syn.to_csv(os.path.join(output, "wellbeing_synergy.csv"), header=["wb_synergy"], encoding="utf-8")
    print("wellbeing_synergy.csv creado (features que más ayudan/perjudican WB).")
else:
    print("Sinergia de bienestar: aún sin datos suficientes.")

#resumen = narrative_summary(df, targets)
resumen = descriptive_summary(df, targets,row_idx=-1)
resumen